# Agent0 Tutorial


Agent0 provides a self contained Hyperdrive simulator using Interactive Local Hyperdrive. It works by managing a local Anvil
instance with an interactive and customizable Hyperdrive deployment, a data collection service, and a dashboard server
showing useful information for analysis.

To start, follow the installation instructions outlined [here](../INSTALL.md) for interactive hyperdrive.


## Simulating trades using Interactive Hyperdrive

The following code initializes a local interactive hyperdrive with
(1) a local chain,
(2) a deployed hyperdrive pool on the local chain, and
(3) a funded agent attached to the pool ready to trade.


In [1]:
import datetime
from fixedpointmath import FixedPoint
from agent0 import LocalHyperdrive, LocalChain

chain = LocalChain()
hyperdrive = LocalHyperdrive(chain)
hyperdrive_agent0 = hyperdrive.init_agent(base=FixedPoint(100_000))

After initializing, we can use the `hyperdrive_agent0` object to simulate trades on the deployed pool.


In [2]:
open_long_event = hyperdrive_agent0.open_long(base=FixedPoint(100))
open_short_event = hyperdrive_agent0.open_short(bonds=FixedPoint(100))
add_liquidity_event = hyperdrive_agent0.add_liquidity(base=FixedPoint(100))

The output of these trades represents the corresponding emitted event from Hyperdrive. For example, the open long event
is shown below. See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/event_types/index.html) for
documentation on the output event types.


In [3]:
open_long_event

OpenLong(trader='0x888Fe86DEBcceA6CE844B9b57340CA2e4045581a', asset_id=452312848583266388373324160190187140051835877600158453279131187532625774656, maturity_time=1715112000, base_amount=FixedPoint("100.0"), vault_share_amount=FixedPoint("99.999994292237768785"), as_base=True, bond_amount=FixedPoint("100.094931446527329167"), __name__='OpenLong')

We can use Interactive Hyperdrive to simulate advancing time and closing a position. For example, we can simulate
closing the long after a day.


In [4]:
# Advance time for a day
chain.advance_time(datetime.timedelta(days=1))
close_long_event = hyperdrive_agent0.close_long(
    maturity_time=open_long_event.maturity_time,
    bonds=open_long_event.bond_amount,
)

## Analyzing Hyperdrive

Interactive hyperdrive provides a fully managed data service that collects information from the chain, as
well as providing an interactive dashboard to view information on the pool. Running the dashboard within Interactive
Hyperdrive brings up a webpage that shows you basic information of the trades made on the pool, as well as information
on a specific agent, including the value of the portfolio. You can also view the dashboard directly in your Jupyter notebook.


In [5]:
dashboard = hyperdrive.get_dashboard_iframe()
display(dashboard)


  You can now view your Streamlit app in your browser.

  URL: http://localhost:7777

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


For customized analysis, Interactive Hyperdrive exposes the underlying data itself. See the `get_*` functions defined in
interactive hyperdrive
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/interactive_hyperdrive/index.html)
for more information.


## Trading policies

Agent0 introduces agent policies, which allows an agent to make trades based on some defined behavior.
Agent0 provides a set of ready-to-use policies. For example, the `random` policy simply makes random trades
on the pool.


In [6]:
from agent0 import PolicyZoo

print(PolicyZoo.random.description())

  A simple demonstration agent that chooses its actions randomly.
  It can take 7 actions: open/close longs and shorts, add/remove liquidity, and redeem withdraw shares.
  Trade size is randomly drawn from a normal distribution with mean of 10% of budget and standard deviation of 1% of budget.
  A close action picks a random open position of the given type (long or short) and attempts to close its entire size.
  Withdrawals of liquidity and redemption of withdrawal shares is sized the same as an open position: N(0.1, 0.01) * budget.


We can attach a policy when initializing an agent and execute the underlying policy using
interactive hyperdrive. The output is then a list of output events corresponding with the
list of actions the policy took in that step. In the example below, we create a random agent
and execute one iteration of the policy, then printing out the trade the random agent made.


In [7]:
random_agent = hyperdrive.init_agent(
    base=FixedPoint(1_000),
    policy=PolicyZoo.random,
    policy_config=PolicyZoo.random.Config(rng_seed=123),
)
random_agent.execute_policy_action()

[OpenShort(trader='0x35DD4CBFD0c3D6bcBAFAC4fb70BEef0882A86Bb7', asset_id=904625697166532776746648320380374280103671755200316906558262375063536523712, maturity_time=1715198400, base_amount=FixedPoint("0.110312209275189716"), vault_share_amount=FixedPoint("0.110297079098986197"), as_base=True, base_proceeds=FixedPoint("112.770034371258485699"), bond_amount=FixedPoint("112.879252612892491776"), __name__='OpenShort')]

See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/policies/index.html#)
for a list and description of all implemented policies.


### Custom Policies

Agent0 also provides an interface to write a custom policy. In this tutorial, we'll write a
policy that simply opens a long if the fixed rate is above a threshold.
Similarly to existing policies, this policy can then be attached to interactive hyperdrive to
simulate the policy within the simulator.

We'll start by defining the class that subclasses from the base policy and define a custom policy
configuration that defines the threshold. We need to overwrite the `action` method in the
policy, which defines what actions the policy takes. In the example below, the agent will simply
open a single long for `open_long_amount` when the fixed rate reaches the `fixed_rate_threshold`,
and close the long at maturity.


In [8]:
# Relevant imports
from agent0 import HyperdriveBasePolicy, open_long_trade, close_long_trade
from dataclasses import dataclass

In [9]:
class OpenLongPolicy(HyperdriveBasePolicy):
    @dataclass(kw_only=True)
    class Config(HyperdriveBasePolicy.Config):
        fixed_rate_threshold: FixedPoint
        open_long_amount: FixedPoint

    def action(self, interface, wallet):
        # Defines if the bot is done trading. We expect this bot to run continuously,
        # so this is always false.
        done_trading = False
        # If no longs in wallet, we check our fixed rate threshold and open the long if threshold reached.
        if len(wallet.longs) == 0:
            if interface.calc_spot_rate() > self.config.fixed_rate_threshold:
                return [open_long_trade(self.config.open_long_amount)], done_trading
        # If there are longs in the wallet, we check for maturity and close them if maturity reached.
        else:
            for maturity_time, long in wallet.longs.items():
                if interface.get_block_timestamp(interface.get_current_block()) >= maturity_time:
                    return [close_long_trade(long.balance, maturity_time)], done_trading
        # We don't do any trades otherwise.
        return [], done_trading

See [here](custom_policy.py) for a more in depth example of writing a policy.


### Running the trading policy in Interactive Hyperdrive

We can test our new policy using interactive hyperdrive before running it on an actual chain. We can do this by
attaching our new policy to a new agent, while passing in a fixed rate threshold of 6%.


In [10]:
policy_agent = hyperdrive.init_agent(
    base=FixedPoint(1_000_000),
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)

Lets take a look at the fixed rate after the set of trades we've made. We can see below that the current fixed rate sits
right around 5%.


In [11]:
hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-04-30 20:23:38,0.049999999999999996
1,21,2024-04-30 20:23:50,0.049999999999999996
2,22,2024-04-30 20:24:02,0.049999999999999996
3,23,2024-04-30 20:24:14,0.049999939182615754
4,24,2024-04-30 20:24:26,0.049999999941576964
5,25,2024-04-30 20:24:38,0.049999999941576964
6,73,2024-05-01 20:24:50,0.049999999941576964
7,74,2024-05-01 20:25:02,0.050000052064751971
8,75,2024-05-01 20:25:14,0.050000052064751971
9,76,2024-05-01 20:25:26,0.050000052064751971


We can now execute our new agent's policy. Since the fixed rate is below the threshold, we expect the agent to not make
any trades, denoted by the output of `execute_policy_action` being an empty list.


In [12]:
policy_agent.execute_policy_action()

[]

Let's make a trade to push the fixed rate to be 6%.


In [13]:
policy_agent.open_short(bonds=FixedPoint(20_000_000))
hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-04-30 20:23:38,0.049999999999999996
1,21,2024-04-30 20:23:50,0.049999999999999996
2,22,2024-04-30 20:24:02,0.049999999999999996
3,23,2024-04-30 20:24:14,0.049999939182615754
4,24,2024-04-30 20:24:26,0.049999999941576964
5,25,2024-04-30 20:24:38,0.049999999941576964
6,73,2024-05-01 20:24:50,0.049999999941576964
7,74,2024-05-01 20:25:02,0.050000052064751971
8,75,2024-05-01 20:25:14,0.050000052064751971
9,76,2024-05-01 20:25:26,0.050000052064751971


Once again, let's execute our policy and view the output. We can see that our policy opened a new long because the
fixed rate is above 6%.


In [14]:
policy_agent.execute_policy_action()

[OpenLong(trader='0xA503fD85129E41E0A31e0C607658971e977fA7f7', asset_id=452312848583266388373324160190187140051835877600158453279131187532625861056, maturity_time=1715198400, base_amount=FixedPoint("100000.0"), vault_share_amount=FixedPoint("99986.276614078975487391"), as_base=True, bond_amount=FixedPoint("100119.747042597919407309"), __name__='OpenLong')]

## Connecting to a remote chain

Along with a fully managed simulator, interactive hyperdrive also provides an interface for connecting to any existing remote chain and deployed hyperdrive pool. We can use this interface to make trades or execute any policy on the deployed hyperdrive pool.

In this tutorial, we'll use the Anvil node and Hyperdrive pool being hosted by the `LocalChain`
and `LocalHyperdrive` objects, but we can simply replace the corresponding configurations with any valid RPC and contract addresses.


In [15]:
from agent0 import Chain, Hyperdrive

# Get the RPC address and the hyperdrive contract addresses to connect to from the managed
# interactive hyperdrive objects.
rpc_uri = chain.rpc_uri
hyperdrive_address = hyperdrive.get_hyperdrive_address()

# We can specify other parameters as such.
# rpc_uri = "<rpc_uri>"
# hyperdrive_address = "0x<hyperdrive_address>"

# Connect to the remote chain and hyperdrive objects.
remote_chain = Chain(rpc_uri)
remote_hyperdrive = Hyperdrive(remote_chain, hyperdrive_address)

We now need to initialize an agent by providing a private key to a wallet. In this tutorial,
we'll simply use a random private key to initialize a new wallet, then fund the agent using
the provided `add_funds` method.

<div class="alert alert-block alert-info"> 
NOTE:: The <code>add_funds</code> method calls the <code>anvil_setBalance</code> to fund Eth and 
<code>mint</code> on the base token contract to fund the wallet, which is only possible due to the 
remote chain and base token contract supporting such calls. It's up to the user to ensure the wallet 
associated with the private key is funded when running on a non-test chain.
</div>

<div class="alert alert-block alert-danger"> 
WARNING:: This tutorial generates a random private key and funds this wallet. In practice, the private
key should be passed into the script via an environment variable secret, and one should limit access
to the generated agent object to only the owner of the private key.
</div>

We also attach our custom agent as well to allow the agent to execute the policy we wrote earlier.


In [16]:
from agent0.core.base.make_key import make_private_key

remote_agent = remote_hyperdrive.init_agent(
    private_key=make_private_key(),
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)
# Add funds to the agent
remote_agent.add_funds(base=FixedPoint(1_000_000), eth=FixedPoint(100))
# Give the hyperdrive contract max approval to withdrawal funds from the remote agent
# TODO need to add setting a custom approval amount
remote_agent.set_max_approval()

This agent is now able to make trades on the remote chain, as well as executing the underlying policy


In [17]:
remote_agent.open_short(bonds=FixedPoint(200))

OpenShort(trader='0x88d61CAc63D7057Bb73d02b022D6c87ecbc1d4Eb', asset_id=904625697166532776746648320380374280103671755200316906558262375063536523712, maturity_time=1715198400, base_amount=FixedPoint("0.24586616185096029"), vault_share_amount=FixedPoint("0.245832406657190211"), as_base=True, base_proceeds=FixedPoint("199.756098771479606786"), bond_amount=FixedPoint("200.0"), __name__='OpenShort')

In [18]:
remote_agent.execute_policy_action()

[OpenLong(trader='0x88d61CAc63D7057Bb73d02b022D6c87ecbc1d4Eb', asset_id=452312848583266388373324160190187140051835877600158453279131187532625861056, maturity_time=1715198400, base_amount=FixedPoint("100000.0"), vault_share_amount=FixedPoint("99986.269004773767680122"), as_base=True, bond_amount=FixedPoint("100119.612148284479444947"), __name__='OpenLong')]

We find that the wallet contains both the short we opened manually and the long we opened using
the policy.


In [19]:
remote_agent.wallet

HyperdriveWallet(address=HexBytes('0x88d61cac63d7057bb73d02b022d6c87ecbc1d4eb'), balance=Quantity(amount=FixedPoint("899999.75413383814903971"), unit=<TokenType.BASE: 'base'>), lp_tokens=FixedPoint("0.0"), withdraw_shares=FixedPoint("0.0"), longs={1715198400: Long(balance=FixedPoint("100119.612148284479444947"), maturity_time=1715198400)}, shorts={1715198400: Short(balance=FixedPoint("200.0"), maturity_time=1715198400)})

## Cleanup

Recall that the local chain created at the beginning of this tutorial produces resources under
the hood that need to be cleaned up. The call below releases all the resources used by
interactive hyperdrive.

**Note**: This cell will terminate process for the database, local chain, and the dashboard running above.


In [20]:
chain.cleanup()

## Final words

In conclusion, this tutorial showcases the interactive hyperdrive workflow. We first explored
the simulation Hyperdrive environment and the analysis tools it provides. We then explored
trading policies and wrote a quick example of a custom policy, and how they plug into the
interactive hyperdrive ecosystem. Finally, we showed how to execute trades and policies
on any remote hyperdrive deployment.

Happy trading!
